In [3]:
import pdfplumber
import pandas as pd
from langdetect import detect

pdf = pdfplumber.open('../example_pdfs/N25610-56_9707_11_OCT19_Russelhobbs.pdf')
pages = pdf.pages

fontname = 'FWYLUD+MyriadPro-Bold'
headline1_size = 12
headline2_size = 9


In [17]:
headline1 = None
headline2 = None

document_pages = []

for page in pages[5:6]:
    page_rows = []
    rows = page.extract_words(extra_attrs=['fontname', 'size'], keep_blank_chars=True)
    for row in rows:
        row_dic = {}
        
        if row['fontname'] == fontname and row['size'] == headline1_size and row['text'].isupper():
            headline1 = row['text']
            headline2 = None
            continue
        
        if row['fontname'] == fontname and row['size'] == headline2_size and row['text'].isupper():
            headline2 = row['text']
            continue
        
        if not headline1:
            continue 
        
        if len(row['text']) < 2:
            continue
         
        row_dic['header1'] = headline1
        row_dic['header2'] = headline2
        row_dic['paragraph'] = row['text']
        
        page_rows.append(row_dic)
    
    document_pages.append(page_rows)
        
        

In [3]:
document_pages[0][100]

{'header1': 'USING GRIND & BREW MODE', 'header2': None, 'paragraph': 'g'}

In [15]:
df_test = pd.DataFrame(document_pages[0])

In [16]:
df_test.to_csv('russelhobbs_output.csv')

In [10]:
df_test[df_test['header1'].str.contains('OPERATING MODES')]['paragraph'].values.tolist()

[]

In [12]:
df_test.groupby(['header1', "header2"]).agg({'paragraph': ' '.join})

paragraph
header1              header2                                                                
CARE AND MAINTENANCE CLEANING THE GRINDER  The grinder can be disassembled in order to cl...
                     GENERAL CLEANING      1.  Unplug the appliance and let it cool. 2.  ...
                     SELF CLEAN            After 60 brewing cycles, the display will show...

In [7]:
page = pdf.pages[3]

In [8]:
words = page.extract_words(extra_attrs=['fontname', 'size'], keep_blank_chars=True)

In [9]:
text = page.extract_text()

In [10]:
df = pd.DataFrame(words)

In [11]:
df

,text,x0,x1,top,doctop,bottom,upright,direction,fontname,size
0,"Read the instructions, keep them safe, pass th...",39.6850,428.91997,39.6852,2565.3552,48.6852,True,1,SIIGUD+MyriadPro-Regular,9.0
1,IMPORTANT SAFEGUARDS,39.8050,183.35500,60.3391,2586.0091,72.3391,True,1,FWYLUD+MyriadPro-Bold,12.0
2,"Follow basic safety precautions, including:",39.6850,182.87437,76.2237,2601.8937,85.2237,True,1,SIIGUD+MyriadPro-Regular,9.0
3,This appliance can be used by persons with red...,39.6850,547.32109,88.2237,2613.8937,161.4238,True,1,SIIGUD+MyriadPro-Regular,13.0
4,h,39.6850,54.27490,161.8258,2687.4958,174.8258,True,1,DRTPSH+MyriadISOtext,13.0
...,...,...,...,...,...,...,...,...,...,...
62,or,317.4106,327.99325,695.9167,3221.5867,704.9167,True,1,SIIGUD+MyriadPro-Regular,9.0
63,min,327.9930,342.04488,695.9167,3221.5867,704.9167,True,1,FWYLUD+MyriadPro-Bold,9.0
64,"buttons within 15 seconds, the coffee maker w...",342.0471,545.86578,695.9167,3221.5867,704.9167,True,1,SIIGUD+MyriadPro-Regular,9.0
65,showing the previously set time.,57.6850,167.45053,706.7167,3232.3867,715.7167,True,1,SIIGUD+MyriadPro-Regular,9.0


In [12]:
df_header1 = df[df['fontname'] == 'FWYLUD+MyriadPro-Bold']

In [13]:
df_header2 = df[(df['fontname'] == 'FWYLUD+MyriadPro-Bold') & (df['size'] == 9)]

In [14]:
headlines1 = df_header1[df_header1['size'] == 12].groupby(['top']).agg({'text': ' '.join})

In [15]:
headlines2 = df_header2[df_header2['size'] == 9].groupby(['top']).agg({'text': ' '.join})

In [16]:
headlines2

,text
top,
674.3166,hr min
695.9167,hr min hr min


In [17]:
headlines1.values.tolist()[2][0]

'BEFORE USING FOR THE FIRST TIME'

In [18]:
headline_dict = dict()

for page in pdf.pages:
    words = page.extract_words(extra_attrs=['fontname', 'size'])
    df = pd.DataFrame(words)
    df_bolds = df[df['fontname'] == 'FWYLUD+MyriadPro-Bold']
    
    page_headlines = df_bolds[df_bolds['size'] == 12].groupby(['top']).agg({'text': ' '.join}).values.tolist()
    
    if page_headlines:
        headline_dict[page.page_number] = page_headlines

In [19]:
headline_dict

{4: [['IMPORTANT SAFEGUARDS'],
  ['PARTS'],
  ['BEFORE USING FOR THE FIRST TIME'],
  ['CORD STORAGE'],
  ['SET THE CLOCK']],
 5: [['CONTROLS AND DISPLAY'],
  ['OPERATING MODES'],
  ['USING GRIND & BREW MODE'],
  ['USING PRE-GROUND COFFEE MODE']],
 6: [['KEEP WARM'],
  ['USING THE TIMER'],
  ['A QUICK CUP'],
  ['CARE AND MAINTENANCE']],
 7: [['ERROR CODES'], ['RECYCLING']],
 8: [['WICHTIGE SICHERHEITSHINWEISE'],
  ['BESTANDTEILE'],
  ['VOR ERSTMALIGEM GEBRAUCH'],
  ['KABELAUFBEWAHRUNG'],
  ['DIE UHR STELLEN']],
 9: [['STEUERUNG UND BEDIENFELD'],
  ['BETRIEBSFUNKTIONEN'],
  ['SO VERWENDEN SIE DIE FUNKTION MAHLEN & BRÜHEN'],
  ['SO VERWENDEN SIE DIE FUNKTION KAFFEE AUS GEMAHLENEN BOHNEN BRÜHEN']],
 10: [['WARMHALTEN'],
  ['SO VERWENDEN SIE DEN TIMER'],
  ['EINE SCHNELLE TASSE'],
  ['PFLEGE UND WARTUNG']],
 11: [['FEHLERCODES'], ['RECYCLING']],
 12: [['PRÉCAUTIONS IMPORTANTES'],
  ['PIÈCES'],
  ['RANGEMENT DU CÂBLE'],
  ['RÉGLAGE DE L’HORLOGE']],
 13: [['COMMANDES ET AFFICHAGE'],
  ['MODES

In [2]:
pdf = pdfplumber.open('example_pdfs/ESAM04110B-167807_DeLonghi.pdf')

In [5]:
page = pdf.pages[2]

In [6]:
page.extract_words(extra_attrs=['fontname', 'size'])

[{'text': 'IMPORTANT',
  'x0': 28.4564,
  'x1': 74.3913,
  'top': 28.346900000000005,
  'doctop': 1218.8989,
  'bottom': 39.346900000000005,
  'upright': True,
  'direction': 1,
  'fontname': 'TVKEUZ+MyriadPro-BoldCond',
  'size': 11.0},
 {'text': 'SAFEGUARDS',
  'x0': 76.3603,
  'x1': 126.91629999999998,
  'top': 28.346900000000005,
  'doctop': 1218.8989,
  'bottom': 39.346900000000005,
  'upright': True,
  'direction': 1,
  'fontname': 'TVKEUZ+MyriadPro-BoldCond',
  'size': 11.0},
 {'text': 'Any',
  'x0': 216.8504,
  'x1': 227.4164,
  'top': 28.346900000000005,
  'doctop': 1218.8989,
  'bottom': 37.346900000000005,
  'upright': True,
  'direction': 1,
  'fontname': 'TVKEUZ+MyriadPro-Cond',
  'size': 9.0},
 {'text': 'other',
  'x0': 229.2884,
  'x1': 244.17440000000002,
  'top': 28.346900000000005,
  'doctop': 1218.8989,
  'bottom': 37.346900000000005,
  'upright': True,
  'direction': 1,
  'fontname': 'TVKEUZ+MyriadPro-Cond',
  'size': 9.0},
 {'text': 'use',
  'x0': 246.0464,
  'x1':